![imagenes](logo.png)

# Introducción a Keras
Vamos a ver cómo usar Keras para crear modelos de Deep Learning así como sus funcionalidades básicas.

In [ ]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (10, 10)

### Cargamos los datos


In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler

data = load_breast_cancer()
X, y = data.data, data.target

X = data.data[:,:4]
X_std = StandardScaler().fit_transform(X)

y = y.reshape(569,1)

In [ ]:
y[:20]

Keras por si mismo no se encarga de hacer todoas las operaciones de bajo nivel (operaciones matriciales), sino que soporta varios backends (el motor que hará el entrenamiento), podemos elegir el que queremos activando la variable de entorno `KERAS_BACKEND`.

Keras soporta los siguientes backends:

- [theano](http://deeplearning.net/software/theano/): Librería de deep learning original de python para deep learning. Hoy en dia raramente se usa por si sola.
- [tensorflow](http://www.tensorflow.org/): Librería de deep learning desarrollada por google. 
- [CNTK](https://www.microsoft.com/en-us/cognitive-toolkit/) Librería de deep learning desarrollada por Microsoft

In [ ]:
# !pip install Theano

In [ ]:
import os

os.environ["KERAS_BACKEND"] = "theano" #tensorflow

![imagenes](im27.png)

Ésta red neuronal se implementa facilmente con Keras, usando la clase `Sequential`, que es similar a la clase `RedNeuronal` que implementamos a mano. Simplemente admite un conjunto de capas.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense


modelo = Sequential()

modelo.add(Dense(units=5, activation='sigmoid', input_shape=(4,)))
#modelo.add(Dense(units=3, activation='sigmoid'))
modelo.add(Dense(units=1, activation='sigmoid'))

Alternativamente podemos crear el modelo con las capas directamente

In [ ]:
modelo = Sequential([
    Dense(units=5, activation='sigmoid', input_dim=4),
    Dense(units=1, activation='sigmoid')
])

Ahora solo queda compilar el modelo y ya quedará preparado para entrenar. A la hora de compilar tenemos que definir la función de pérdidas que medirá el error propagado. 


Keras tambien nos permite especificar métricas que calculará para cada batch de entrenamiento y nos las dará como un historial despues de entrenar

Podemos añadir el optimizador como string si queremos usarlo con sus hiperparámetros por defecto (es decir, no queremos modificar su ratio de aprendizaje o cualquier otro hiperparámetro).

In [ ]:
modelo.compile(loss="binary_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

Si queremos modificar los parámetros del optimizador tenemos que crear el objeto optimizador. Keras soporta SGD pero tambien muchos otros.

In [ ]:
from keras.optimizers import SGD

sgd = SGD(lr=0.01)

In [ ]:
modelo.compile(loss="binary_crossentropy",
              optimizer=sgd,
              metrics=["accuracy"])

Podemos ver una descripción del modelo con `summary`

In [ ]:
modelo.summary()

Vemos que tiene 31 parametros para entrenar, esto se corresponde con los pesos de la red.

(4x5 + 5bias  + 5x1 + 1 = 31 pesos)

Ahora podemos ajustar el modelo a los datos de entrenamiento con el método `fit`.  Es importante notar que por defecto keras hace **mini batch**, es decir, no entrena con observaciones individuales, sino con grupos de observaciones (definido el tamaño de los grupos con el parámetro `batch_size`)

In [ ]:
modelo.fit?

In [ ]:
historial = modelo.fit(X_std, y , epochs=100)

Podemos ver la evolución del funcionamiento del modelo desde el historial de entrenamiento

In [ ]:
historial

In [ ]:
plt.plot(historial.history["accuracy"])
plt.title("Exactitud vs épocas de entrenamiento");

Si queremos que no imprima los logs, podemos pasarle al método `fit` el argumento `verbose=0`

Ahora podemos usar el método `predict` como si fuese un estimador de scikit-learn

In [ ]:
modelo.predict(X_std)[:5]

O si queremos predecir las clases directamente podemos usar `predict_classes`

In [ ]:
modelo.predict_classes(X_std)[:5]

Podemos evaluar el funcionamiento del modelo usando `evaluate`

In [ ]:
scores = modelo.evaluate(X_std, y)
scores

In [ ]:
modelo.metrics_names

### Callbacks

`keras` soporta callbacks, que son simplemente funciones que podemos hacer que se ejecuten en cada paso del proceso de entrenamiento. 

Básicamente son clases que heredan de `keras.callbacks.Callback`, con los siguientes métodos disponibles:

- `on_train_begin()` : se ejecuta al iniciar el entrenamiento
- `on_batch_begin()`: se ejecuta al empezar el entrenamiento de un batch (mini batch)
- `on_batch_end()`: se ejecuta al acabar un batch (mini batch)
- `on_epoch_begin()`: se ejecuta al empezar una época de entrenamiento
- `on_epoch_end()`: se ejecuta al acabar una época de entrenamiento

Por ejemplo, Keras calcula lás métricas en cada batch, supongamos que queremos calcular una métrica por época (que es más representativo que hacerlo en un batch)

In [ ]:


from keras.callbacks import Callback
from sklearn.metrics import f1_score, precision_score, recall_score

class MetricasEpoca(Callback):
    def on_train_begin(self, logs={}):
        self.f1_epoca = []
        self.recall_epoca = []
        self.precision_epoca = []
 
    def on_epoch_end(self, epoch, logs={}):
        val_predict = self.model.predict_classes(self.validation_data[0])
        val_targ = self.validation_data[1]
        f1 = f1_score(val_targ, val_predict)
        recall = recall_score(val_targ, val_predict)
        precision = precision_score(val_targ, val_predict)
        self.f1_epoca.append(f1)
        self.recall_epoca.append(recall)
        self.precision_epoca.append(precision)
        
        
modelo = Sequential([
    Dense(units=5, activation='sigmoid', input_dim=4),
    Dense(units=1, activation='sigmoid')
])
modelo.compile(loss='binary_crossentropy', optimizer=sgd)

metricas_epoca = MetricasEpoca()

modelo.fit(X_std, y, validation_data=(X_std, y),
           epochs=50, verbose=0, callbacks=[metricas_epoca]);

In [ ]:
plt.plot(metricas_epoca.f1_epoca)
plt.title("Metrica F1 vs numero de epocas");

### Early Stopping

El entrenamiento de un modelo de deep learning es iterativo, esto significa que en teoría podemos dejar el modelo aprendiendo indefinidamente. En el caso de usar descenso estocástico de gradiente (SGD) para aprender, el error simplemente continuará dando vueltas alrededor del mínimo error.

Para evitar tener que entrenar durante el número definido de épocas si el modelo ya ha convergido antes al mínimo de error, podemos implementar lo que se llama `early stopping`. Básicamente, esto para el entrenamiento cuando se cumplen ciertas condiciones

In [ ]:
from keras.callbacks import EarlyStopping

Los parámetros principales del EarlyStopping son los siguientes:
- **monitor**: La métrica a monitorizar
- **min_delta**: la mínima cantidad de variación entre épocas de la métrica para considerarlo un progreso (y continuar entrenando)
- **patience**: número de épocas sin mejora despues de las cuales se para el entrenamiento

In [ ]:
earlystop = EarlyStopping(monitor='accuracy', min_delta=0.00001, patience=10,
                          verbose=1, mode='auto')


modelo = Sequential([
    Dense(units=5, activation='sigmoid', input_dim=4),
    Dense(units=1, activation='sigmoid')
])
modelo.compile(loss='binary_crossentropy', optimizer=sgd, metrics=["accuracy"])

modelo.fit(X_std, y, epochs=100, 
           verbose=1, callbacks=[earlystop]);

### Guardado de modelos en Keras.

En Keras, podemos guardar un modelo (de forma similar a como haciamos con `joblib/pickle` en `scikit-learn` tanto durante el proceso de entrenamiento (checkpoints) como al acabar el entrenamiento

In [ ]:
modelo.save

In [ ]:
from keras.callbacks import ModelCheckpoint

In [ ]:
checkpoint = ModelCheckpoint(filepath='modelo.hdf5', verbose=1, period=10)


modelo = Sequential([
    Dense(units=5, activation='sigmoid', input_dim=4),
    Dense(units=1, activation='sigmoid')
])
modelo.compile(loss='binary_crossentropy', optimizer=sgd, metrics=["acc"])

nvo_modelo = modelo.fit(X_std, y, epochs=100, 
           verbose=1, callbacks=[checkpoint]);

In [ ]:
!dir

Ahora podemos recargar el modelo guardado con `load_model`

In [ ]:
from keras.models import load_model

In [ ]:
modelo_recargado = load_model("modelo.hdf5")

In [ ]:
modelo_recargado.predict(X_std)[:5]

### Validación Cruzada

Podemos crear Redes en keras de forma que sean compatibles con [Scikit-learn](https://keras.io/scikit-learn-api/)

En Deep Learning, en  general no se suele hacer validación cruzada a menos que el dataset sea pequeño, ya que los tiempos de entrenamiento de modelos y los datasets suelen ser bastante elevados. No obstante si podemos permitirnoslo es aconsejable.

In [ ]:
from sklearn.model_selection import StratifiedKFold

def generar_modelo():
    modelo = Sequential()
    modelo.add(Dense(units=5, activation='sigmoid', input_dim=4))
    modelo.add(Dense(units=1, activation='sigmoid'))
    learning_rate = 0.01
    sgd = SGD(lr=learning_rate)
    modelo.compile(loss="binary_crossentropy",
              optimizer=sgd,
              metrics=['accuracy'])
    return modelo

kfold = StratifiedKFold()
cvscores = []
for train, test in kfold.split(X_std, y):
    modelo = generar_modelo()
    modelo.fit(X_std[train], y[train], epochs=100, verbose=0)
    scores = modelo.evaluate(X_std[test], y[test], verbose=0)
    cvscores.append(scores[1] )

In [ ]:
StratifiedKFold?

In [ ]:
cvscores

In [ ]:
np.mean(cvscores)

## Optimización de hiperparámetros

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier

def generar_modelo(n_oculta=5,activacion="sigmoid"):
    modelo = Sequential()
    modelo.add(Dense(units = n_oculta, activation = activacion, input_dim = 4))
    modelo.add(Dense(units = 1, activation = "sigmoid"))
    sgd = SGD(lr=0.0001)
    modelo.compile(loss="binary_crossentropy",optimizer = sgd,metrics=["accuracy"])
    return modelo

modelo = KerasClassifier(build_fn=generar_modelo,verbose = 0)

También vamos a añadir una variable de entorno para controlar cómo funciona la búsqueda de malla. Generalmente pasamos el argumento `n_jobs=-1` que le indica a sklearn que puede usar todos los núcleos de nuestro ordenador. Esto puede dar problemas al ejecutar el código desde Jupyter.  

In [ ]:
os.environ["JOBLIB_START_METHOD"] = "forkserver"

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

param_grid = {
    "epochs" : [10,30],
    "n_oculta" : [5,20],
    "activacion" : ["sigmoid","relu"]
    
}

grid = GridSearchCV(estimator = modelo, param_grid = param_grid, scoring = "accuracy")
grid_result = grid.fit(X_std,y)

In [ ]:
grid_result

In [ ]:
grid_result.predict(X_std)

In [ ]:
print("Mejor estimador (error {:.5f}): {}".format(grid_result.best_score_,
                                                  grid_result.best_params_))